# Парсинг информации о пользователях vk.com

Важным моментом является то, что в рамках данного программного модуля будет рассмотрена именно задача "парсинга", т.е. задача сбора инфомрации из профилей пользователей и групп социальных сетей.

В ходе работы не будут затронуты функции, связанные с изменением информации в профилях пользователей или способы генерации контента: генерация и постинг сообщений в лентах пользователей, лайк-спам (автоматическое расставление лайков на записи из новостной ленты пользователей социальной сети vk.com) и так далее.

Кроме прочего, социальная сеть vk.com накладывает ряд ограничений на доступ к таким данным, как сообщения пользователей, скрытые друзья или заметки пользователей и пр. Получение таких данных возможно только при наличии определенных прав доступа для API-клиента, которыми данный программный модуль, являясь третьей стороной, не обладает. Ввиду этого ограничения, некотороые данные остались недоступны и в рамках работы и анализа применяться не будут.

## Установка библиотеки для парсинга:
Строки из первого блока необходимо раскоментировать при первом запуске программы

In [1]:
#!pip install vk

In [1]:
import vk;

## Авторизации в vk.com
 - @vk_login – логин для входа в vk.com
 - @vk_password – пароля для входа в социальную сеть vk.com 
 - @app_id – идентификационный номер приложения, которое можно создать в разделе для разработчиков (требуется для ряда методов)
 - @vk_private_key – секретный ключ созданного приложения
 - @vk_service_key – сервисный ключ созданного приложения
 
 Данные вносятся как значения переменных, которые используются далее в коде программы.

In [3]:
vk_login = '89779814062';
vk_password = '01350905Ram';
app_id = '6431562';
vk_private_key = 'JHh2tjrBB61Q5LfEJnNT';
vk_service_key = 'cf7f8d1acf7f8d1acf7f8d1a62cf1dae50ccf7fcf7f8d1a95c141769040f444131e5e2d';


## Получение сессии авторизованного пользователя:
В случае успешной авторизации, под блоком ниже появится сообщение, подтверждающее это. Если авторизация не пройдена или выполнена с некоторой ошибкой, система предупредит об этом.

Название и код ошибки, будут представленны как результат выполнения данного блока

In [8]:
session = vk.AuthSession(app_id=app_id, user_login=vk_login, user_password=vk_password);
vkapi = vk.API(session, v='5.74', lang='ru', timeout=60)

if (vkapi.users.get(user_id=1)):
    print ('Авторизация пройдена успешно')
else:
    print ('При авторизации возникла ошибка, проверь данные из блока Авторизация')

Авторизация пройдена успешно


## Основные функции для работы с API vk.com

### Метод Execute
Особый метод, который позволяет передавать в своем теле запроса другие методы, таким образом комбинируя их:
- позволяет обойти ограничения на обращения к API vk.com
- позволяет задействовать мощность и память аппаратного обеспечения компании Вконтакте, вместо собственных
- ускоряет процесс сбора данных, т.к. запрсы выполняются между нодами доменного пространства непосредтвенно по имени машины или внутреннему IP-адресу а не с внешнего клиента

*@vkapi* – объект для работы с API VK, который был получен в блоке авторизации<br>
*@code* - строка, код алгоритма в VKScript в формате, похожем на JavaSсript или ActionScript (предполагается совместимость с ECMAScript). Алгоритм должен завершаться командой return %выражение%. Операторы должны быть разделены точкой с запятой. Внутри *code* может содержаться не более 25 обращений к методам API.<br>
<a href="https://vk.com/dev/execute"> Подробнее об Execute</a>

In [5]:
def execute(vkapi, code):
    response = vkapi.execute(code=code)
    if (response):
        return response
    else:
        return print('Ошибка инициализации метода')

### ~~Методы fave~~
Методы этого раздела работают с контентом, который отметил пользователь социальной сети. Сериди методов данного разделда было выбрано *5* основных:
- fave.getLinks – возвращает ссылки, добавленные в закладки текущим
- fave.getPhotos – возвращает фотографии, на которых текущий пользователь поставил отметку "Мне нравится"
- fave.getPosts – возвращает записи, на которых текущий пользователь поставил отметку "Мне нравится"
- fave.getUsers – возвращает список пользователей, добавленных текущим пользователем в закладки
- fave.getVideos – возвращает список видео, на которых текущий пользователь поставил отметку "Мне нравится"


### Методы gifts
Методы этого раздела работают со списком подарков в социальной сети.
- gifts.get – возвращает список полученных подарков пользователя

*@user_id* – идентификатор пользователя, для которого необходимо получить список подарков. Положительное число, по умолчанию идентификатор текущего пользователя<br>
*@count* – количество подарков, которое нужно вернуть. Положительное число<br>
*@offset* – смещение, необходимое для выборки определенного подмножества подарков. Положительное число<br>
<a href="https://vk.com/dev/gifts.get"> Подробнее об gifts.Get</a>

In [6]:
def gifts_get(vkapi, user_id, count, offset):
    response = vkapi.gifts.get(user_id=user_id, count=count, offset=offset)
    if (response):
        print ('Запрос выполнено успешно')
        return response;
    else:
        print('В работе функции возникли ошибки')

### Методы groups
Методы этого раздела работают со списком групп пользтвателя. Среди методов было выбрано *4* наиболее функциональных для данной работы:
- groups.get – возвращает список сообшеств пользователя
- groups.isMember – возвращает информацию о том, является ли пользователь участником указанной группы
- groups.getMembers – возвращает информациб о пользователях группы
- groups.getCatalog – возвращает список сообществ выбранной категории каталога
- groups.getCatalogInfo – возвращает список категорий для каталога сообществ

*@user_id* – идентификатор пользователя, информацию о сообществах которого требуется получить.<br>
*@extended* – если указать в качестве этого параметра 1, то будет возвращена полная информация о группах пользователя. По умолчанию 0<br>
*@filter* – список фильтров сообществ, которые необходимо вернуть, перечисленные через запятую. Доступны значения admin, editor, moder, groups, publics, events. По умолчанию возвращаются все сообщества пользователя. 
При указании фильтра admin будут возвращены сообщества, в которых пользователь является администратором, editor — администратором или редактором, moder — администратором, редактором или модератором. <br>
*@fields* – список дополнительных полей, которые необходимо вернуть. Возможные значения: city, country, place, description, wiki_page, members_count, counters, start_date, finish_date, can_post, can_see_all_posts, activity, status, contacts, links, fixed_post, verified, site, can_create_topic. Подробнее см. по ссылке ниже<br>
*@offset* – смещение, необходимое для выборки определённого подмножества сообществ.<br>
*@count* – количество сообществ, информацию о которых нужно вернуть.<br>



<a href="https://vk.com/dev/groups.get"> Подробнее об groups.Get</a>

In [7]:
def groups_get(vkapi, user_id, extended, filter, fields, offset, count):
    response = vkapi.groups.get(user_id=user_id, extended=extended, filter=filter,  offset=offset, count=count)
    if (response):
        print ('Запрос выполнено успешно')
        return response;
    else:
        print('В работе функции возникли ошибки')

*@group_id* – идентификатор или короткое имя сообщества. <br>
*@user_id* – идентификатор пользователя. <br>
*@user_ids* – идентификаторы пользователей, не более 500.<br>
*@extended* – 1 — вернуть ответ в расширенной форме. По умолчанию — 0. <br>

<a href="https://vk.com/dev/groups.isMember"> Подробнее об groups.isMember</a>

In [8]:
def groups_isMember(vkapi,group_id, user_id, user_ids, extended):
    response = vkapi.groups.isMember(group_id=group_id, user_id=user_id, user_ids=user_ids, extended=extended)
    if (response):
        print ('Запрос выполнено успешно')
        return response;
    else:
        print('В работе функции возникли ошибки')

*@group_id* – идентификатор или короткое имя сообщества. <br>
*@profiles* – количество профилей, которое нужно получить. <br>
*@fields* – данные из профиля, которые скрипт должен вернуть. <br>
*@sort* – режим сотрировки (по возрастанию / убыванию ID-пользователей). <br>

In [9]:
def groups_getMembers (vkapi, group_id, profiles, fields, sort):
    times = profiles/1000
    i = 0;
    offset=0;
    while i < times:
        all_profiles =[];
        profiles = vkapi.groups.getMembers(group_id=group_id, count=1000, fields=fields, offset = offset, sort = sort);
        all_profiles.extend(profiles['items']);
        with open('g_parse/members'+str(i)+'.txt', 'w', encoding='utf-8') as f:
            json.dump(all_profiles, f)
            f.close()
        offset=offset+1000
        i=i+1;
        #sleep(1) // для обхода блокировки API
    return all_profiles;

*@category_id* – идентификатор категории, полученный в методе groups.getCatalogInfo. <br>
*@subcategory_id* – идентификатор подкатегории, полученный в методе groups.getCatalogInfo<br>

<a href="https://vk.com/dev/groups.getCatalog"> Подробнее об groups.GetCatalog</a>

In [9]:
def groups_getCatalog(vkapi, category_id, subcategory_id):
    response = vkapi.groups.getCatalog(category_id=category_id, subcategory_id=subcategory_id)
    if (response):
        print ('Запрос выполнено успешно')
        return response;
    else:
        print('В работе функции возникли ошибки')

*@extended* – 1 — вернуть информацию о количестве сообществ в категории и три сообщества для предпросмотра.<br>
*@subcategories* –  — вернуть информацию об подкатегориях. <br>

<a href="https://vk.com/dev/groups.getCatalogInfo"> Подробнее об groups.GetCatalogInfo</a>

In [10]:
def groups_getCatalogInfo(vkapi, extended, subcategories):
    response = vkapi.groups.getCatalogInfo(extended=extended, subcategories=subcategories)
    if (response):
        print ('Запрос выполнено успешно')
        return response;
    else:
        print('В работе функции возникли ошибки')

### Методы likes
Методы этого раздела работают контентом, который пользователь выделил отметкой "Мне нравится". Данные методы позволяют оценить активность пользователя в социальной сети.
- likes.isLiked – проверяет, находится ли объект в списке "Мне нравится" заданного пользователя

*@user_id* – идентификатор пользователя, у которого необходимо проверить наличие объекта в списке «Мне нравится». Если параметр не задан, то считается, что он равен идентификатору текущего пользователя.<br>
*@type* – тип объекта <br>
*@owner_id* – идентификатор владельца Like-объекта. Если параметр не задан, то считается, что он равен идентификатору текущего пользователя.  <br>
*@item_id* – идентификатор объекта. <br>


<a href="https://vk.com/dev/likes.isLiked"> Подробнее об likes.isLiked</a>

In [11]:
def likes_isLiked(vkapi, user_id, type, owner_id, item_id):
    response = vkapi.likes.isLiked(user_id=user_id, type=type, owner_id=owner_id, item_id=item_id)
    if (response):
        print ('Запрос выполнено успешно')
        return response;
    else:
        print('В работе функции возникли ошибки')

### ~~Методы messages~~
Методы этого раздела работают с сообщениями пользователя. В силу правил безопасности и конфиденциальности, большинство методов данного раздела закрыты для публичного использования и для них требуются особые права доступа. Были выделены *2* полезных для данной работы метода: 
- messages.getLastActivity – возвращает текущий статус и дату последней активности указанного пользователя
- messages.isMessages – возвращает информацию о том, разрешена ли отправка сообщений сообщества пользователю

### Методы newsfeed
Методы этого раздела работают новостной лентой пользователя.
- newsfeed.get – возвращает данные, необходимые для показа списка новостей текущего пользователя
- newsfeed.getBanned – возвращает список пользователей и групп, которые текущий пользователь скрыл из ленты новостей
- newsfeed.getLists – возвращает пользовательские списки новостей
- newsfeed.getRecomm... – получает список новостей, рекомендованных пользователю

In [42]:
### код будет тут

### Методы notes
Методы этого раздела работают со списком заметок пользователя.
- notes.get – возвращает список заметок, созданных пользователем

In [43]:
### код будет тут

### Методы photos
Методы этого раздела работают с фотографиями пользователя. Многие, как и методы messages, не доступны ввиду требований безопасности. Было выбрано *3* ключевых:
- photos.getAlbumsCount – возвращает количество доступных альбомов пользователя или сообщества
- photos.getAll – возвращает все фотографии пользователя или сообщества в антихронологическом порядке
- photos.getUserPhotos – возвращает список фотографий на которых отмечен пользователь


In [44]:
### код будет тут

### Методы places
Методы этого раздела работают с местами, в которых отметился пользователь.
- places.getTypes – возвращает список всех возможных типов мест
- places.getCheckins – возвращает список отметок пользователей в местах согласно заданным параметрам

In [45]:
### код будет тут

### Методы status
Методы этого раздела работают со статусами пользователей
- status.get – получает текст статуса пользователя или сообщества

In [46]:
### код будет тут

### Методы stories
Методы этого раздела работают с историями пользователей в новостной ленте пользователя vk.com
- stories.get – возвращает истории, доступные для текущего пользователя

In [47]:
### код будет тут

### Методы video
Методы этого раздела работают со списком видео-записей пользователя
- video.get – возвращает информацию о видеозаписях
- video.getAlbums – возвращает список альбомов видеозаписей пользователя или сообщества

In [50]:
### код будет тут

### Методы users
Методы этого раздела работают с данными о пользователях социальной сети vk.com. Они позволяют получить данные профиля пользователя, статистику, друзей и пр. данные. Кроме того, методы данного раздела позволяют осуществлять поиск по пользователям социальной сети, получая их ID, которые используются в большинстве вышеописанных методов
- users.get – возвращает расширенную информацию о пользователях
- users.getFollowers – возвращает список идентификаторов пользователей, которые являются подписчиками пользователя
- users.getSubscriptions – возвращает список идентификаторов пользователей и публичных страниц, которые входят в список подписок пользователя
- users.isAppUser – возвращает информацию о том, установил ли пользователь приложение
- users.search – возвращает список пользователей в соответствии с заданным критерием поиска

In [49]:
### код будет тут

### Методы wall
Методы этого раздела работают со списком видео-записей пользователя
- wall.get – возвращает список записей со стены пользователя или сообщества
- wall.getComments – возвращает список комментариев к записи на стене
- wall.getReposts – позволяет получать список репостов заданной записи

In [51]:
### код будет тут

<hr>

# Main функция
Данный раздел включает в себя работу с методами, описанными выше. Для каждого из них были сформированы специальные функции, которые облегчают работу с API vk.com, кроме того, они выводят информацию уже в обработанном виде, который упрощает визуальное восприятие и повышает эффективность процесса feature engeneering

*Для повышения производительности main функции рекомендуется использовать функцию execute, которая комбинирует или мультиплицирует ряд запросов. Кроме того, рекомендуется подключить и использовать пакеты для работы с данными, например pandas, numpy, seaborn и т.д. Примеры пакетов приведены ниже*

In [1]:
import pandas as pd
pd.set_option("display.max_columns", 346)
import numpy as np
from pandas.io.json import json_normalize
import csv;
from time import sleep;
import json;
import re;

In [2]:
import vk;

## Авторизации в vk.com
 - @vk_login – логин для входа в vk.com
 - @vk_password – пароля для входа в социальную сеть vk.com 
 - @app_id – идентификационный номер приложения, которое можно создать в разделе для разработчиков (требуется для ряда методов)
 - @vk_private_key – секретный ключ созданного приложения
 - @vk_service_key – сервисный ключ созданного приложения
 
 Данные вносятся как значения переменных, которые используются далее в коде программы.

In [3]:
vk_login = '89779814062';
vk_password = '01350905Ram';
app_id = '6431562';
vk_private_key = 'JHh2tjrBB61Q5LfEJnNT';
vk_service_key = 'cf7f8d1acf7f8d1acf7f8d1a62cf1dae50ccf7fcf7f8d1a95c141769040f444131e5e2d';


## Получение сессии авторизованного пользователя:
В случае успешной авторизации, под блоком ниже появится сообщение, подтверждающее это. Если авторизация не пройдена или выполнена с некоторой ошибкой, система предупредит об этом.

Название и код ошибки, будут представленны как результат выполнения данного блока

In [4]:
session = vk.AuthSession(app_id=app_id, user_login=vk_login, user_password=vk_password);
vkapi = vk.API(session, v='5.74', lang='ru', timeout=100000000)

if (vkapi.users.get(user_id=1)):
    print ('Авторизация пройдена успешно')
else:
    print ('При авторизации возникла ошибка, проверь данные из блока Авторизация')

Авторизация пройдена успешно


### Шаг 1 – сбор ID пользователей
Для того, чтобы начать работу с API vk.com и использовать все вышеописанные методы, необходимо получить список пользователей с их ID. 
Ниже приведен пример кода, который выполняет эту задачу.

Объявляем общие переменные, которые будет использоваться в дальнейшем в работе с API
- @fields: поля профиля пользователя vk, которые необходимо получить. По-умолчанию запрашиваются все поля пользователя, подробнее о них можно прочитать по <a href = "https://vk.com/dev/objects/user">ссылке</a>

In [6]:
fields = ['photo_id','verified','sex','bdate','city','country','home_town',
          'has_photo','photo_50','photo_100','photo_200_orig','photo_200',
          'photo_400_orig','photo_max','photo_max_orig','online','lists',
          'domain','has_mobile','contacts','site','education','universities',
          'schools','status','last_seen','followers_count','common_count',
          'occupation','nickname','relatives','relation','personal','connections',
          'exports','wall_comments','activities','interests','music','movies',
          'tv','books','games','about','quotes','can_post','can_see_all_posts','can_see_audio',
          'can_write_private_message','can_send_friend_request',
          'is_favorite','is_hidden_from_feed','timezone','screen_name','maiden_name','crop_photo',
          'is_friend','friend_status','career','military','blacklisted','blacklisted_by_me']

### Функция циклического парсинга профилей

- @offset: должен быть меньше @count
- @sort: 1 – по дате регистрации, 0 - по популярности
- @times: количество циклов парсинга
- @count: количество получаемых профилей, не более 1000
- @fields: поля, которые парсер должен запросить. Они определены в массиве @fields выше
- @vkapi: объект авторизации в vk.com

In [7]:
def cycle_parser (vkapi, times, count, fields, offset, sort):
    i = 0;
    all_profiles =[];
    while i < times:
        profiles = vkapi.users.search(count=count, fields=fields, offset = offset, sort = sort);
        only_items = profiles['items'];
        all_profiles.extend(only_items);
        i=i+1;
        #sleep(1) // для обхода блокировки API
    return all_profiles;

### Запуск парсера

В результате мы получаем *i* количество файлов с названием *k_profiles_i*, в кадом из которых содержится около 1000 профилей пользователей

In [10]:
%%time
groups_getMembers(vkapi, 22822305, 1000000, fields, 'id_asc');

CPU times: user 5min 28s, sys: 17.6 s, total: 5min 46s
Wall time: 3h 10min 52s


[{'bdate': '23.9',
  'blacklisted': 0,
  'blacklisted_by_me': 0,
  'can_post': 0,
  'can_see_all_posts': 1,
  'can_see_audio': 0,
  'can_send_friend_request': 1,
  'can_write_private_message': 0,
  'common_count': 0,
  'country': {'id': 2, 'title': 'Украина'},
  'crop_photo': {'crop': {'x': 0.0, 'x2': 100.0, 'y': 0.0, 'y2': 100.0},
   'photo': {'album_id': -6,
    'date': 1524683091,
    'height': 1200,
    'id': 456240680,
    'owner_id': 136791186,
    'photo_1280': 'https://pp.userapi.com/c834104/v834104882/1258e4/WQkAmK0UDy8.jpg',
    'photo_130': 'https://pp.userapi.com/c834104/v834104882/1258e1/-q7OOAQ7Wvw.jpg',
    'photo_2560': 'https://pp.userapi.com/c834104/v834104882/1258e5/T2orvC_PoIk.jpg',
    'photo_604': 'https://pp.userapi.com/c834104/v834104882/1258e2/wRKj24krrCA.jpg',
    'photo_75': 'https://pp.userapi.com/c834104/v834104882/1258e0/haYM9w8794s.jpg',
    'photo_807': 'https://pp.userapi.com/c834104/v834104882/1258e3/yTzKZNn-z_0.jpg',
    'post_id': 423,
    'text': ''

### Нормализация json-ов и удобный импорт в pandas

#### Вариант 1

Следующим шагом, необходимо данные файлы нормализовать (сделать из вложенного json-объекта плоскую таблицу). Для этого используется утилита csvkit. Подробнее об ее установке и использовании по <a href="https://csvkit.readthedocs.io/en/1.0.3/"> ссылке</a>.<br>
Ниже пример консольной команды которая преобразует Json-файл в CSV

In [20]:
#!in2csv -f json -e utf-8 -k ITEMS k_profiles_i.txt > data_i.csv

In [16]:
%%time
path = 'input/'
df1 = pd.read_csv(path+'data.csv', low_memory=False)
df2 = pd.read_csv(path+'data_1.csv', low_memory=False)
df3 = pd.read_csv(path+'data_2.csv', low_memory=False)
df4 = pd.read_csv(path+'data_3.csv', low_memory=False)
df5 = pd.read_csv(path+'data_4.csv', low_memory=False)
df6 = pd.read_csv(path+'data_5.csv', low_memory=False)
df7 = pd.read_csv(path+'data_6.csv', low_memory=False)
df8 = pd.read_csv(path+'data_7.csv', low_memory=False)
df9 = pd.read_csv(path+'data_8.csv', low_memory=False)
df10 = pd.read_csv(path+'data_9.csv', low_memory=False)
df11 = pd.read_csv(path+'data_10.csv', low_memory=False)

frames = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]

CPU times: user 1min 7s, sys: 7.94 s, total: 1min 15s
Wall time: 1min 16s


In [17]:
%%time

data = pd.concat(frames)
data.shape

CPU times: user 15.9 s, sys: 1.54 s, total: 17.4 s
Wall time: 17.6 s


#### Вариант 2
Можно импортировать данные в формате raw-JSON и уже методами pandas и Python, нормализовать его. Рассмотрим подробнее этот вариант: 

In [4]:
%%time
path = 'g_parse/'
profiles = pd.DataFrame()

for i in range(0, 100):
    df = pd.read_json(path+'profiles_'+str(i)+'.txt', 'values')
    profiles = pd.concat([profiles, df], ignore_index=True)

profiles.drop_duplicates(['id'], inplace = True)

CPU times: user 43.6 s, sys: 5.31 s, total: 49 s
Wall time: 50.6 s


### Нормализация данных

Далее, стараемся привести данные к плоской таблице, процесс выполняется в несколько итераций:

#### Итерация 1

In [111]:
%%time
cities_frame = pd.DataFrame()
for city in profiles['city']:
    try:
        cities_frame = cities_frame.append(json_normalize(city), ignore_index=True)
    except:
        cities_frame = cities_frame.append(['NaN'], ignore_index=True)
    

CPU times: user 9min 23s, sys: 10.2 s, total: 9min 33s
Wall time: 9min 36s


In [112]:
cities_frame.drop(0, axis=1, inplace = True)
cities_frame.rename(columns={'id':'city/id', 'title':'city/title'}, inplace=True)
cities_frame.shape

(100000, 2)

In [133]:
result = result.join(cities_frame)
result.shape

ValueError: columns overlap but no suffix specified: Index(['city/id', 'city/title'], dtype='object')

#### Итерация 2

In [108]:
%%time
country_frame = pd.DataFrame()
for coutnry in profiles['country']:
    try:
        country_frame = country_frame.append(json_normalize(coutnry), ignore_index=True)
    except:
        country_frame = country_frame.append(['NaN'], ignore_index=True)
    

CPU times: user 10min 9s, sys: 11.3 s, total: 10min 21s
Wall time: 10min 26s


In [109]:
country_frame.drop(0, axis=1, inplace = True)
country_frame.rename(columns={'id':'country/id', 'title':'country/title'}, inplace=True)
country_frame.shape

(100000, 2)

In [132]:
result = result.join(country_frame)
result.shape

(100000, 138)

#### Итерация 3

In [105]:
%%time
last_seen_frame = pd.DataFrame()
for last_seen in profiles['last_seen']:
    try: 
        last_seen_frame = last_seen_frame.append(json_normalize(last_seen), ignore_index=True)
    except:
        last_seen_frame = last_seen_frame.append(['NaN'], ignore_index=True)

CPU times: user 2min 33s, sys: 1.51 s, total: 2min 35s
Wall time: 2min 35s


In [106]:
last_seen_frame.drop(0, axis=1, inplace = True)
last_seen_frame.rename(columns={'time':'last_seen/time', 'platform':'last_seen/platform'}, inplace=True)
last_seen_frame.shape

(100000, 2)

In [131]:
result = result.join(last_seen_frame)
result.shape

(100000, 136)

#### Итерация 4

In [102]:
%%time
career_frame = pd.DataFrame()
for career in profiles['career']:
    try:
        career_frame = career_frame.append(json_normalize(career[0]), ignore_index=True)
    except:
        career_frame = career_frame.append(['NaN'], ignore_index=True)

career_frame.drop(0, axis=1, inplace = True)

CPU times: user 15min 23s, sys: 38.9 s, total: 16min 2s
Wall time: 16min 10s


In [103]:
col_names=[]
for element in list(career_frame):
    name = 'carreer/0/'+element;
    col_names.append(name)
dic = dict(zip(list(career_frame), col_names))

career_frame.rename(columns=dic, inplace=True)
career_frame.shape

(100000, 7)

In [130]:
result = result.join(career_frame)
result.shape

(100000, 134)

#### Итерация 5

In [99]:
%%time
occupation_frame = pd.DataFrame()
for occupation in profiles['occupation']:
    try:
        occupation_frame = occupation_frame.append(json_normalize(occupation), ignore_index=True)
    except:
        occupation_frame = occupation_frame.append(['NaN'], ignore_index=True)
occupation_frame.drop(0, axis=1, inplace = True)

CPU times: user 20min 24s, sys: 34.5 s, total: 20min 59s
Wall time: 21min 23s


In [100]:
col_names=[]
for element in list(occupation_frame):
    name = 'occupation/'+element;
    col_names.append(name)
dic = dict(zip(list(occupation_frame), col_names))

occupation_frame.rename(columns=dic, inplace=True)
occupation_frame.shape

(100000, 3)

In [129]:
result = result.join(occupation_frame)
result.shape

(100000, 127)

#### Итерация 6

In [74]:
%%time
schools_frame = pd.DataFrame()
for schools in profiles['schools']:
    try:
        schools_frame = schools_frame.append(json_normalize(schools[0]), ignore_index=True)
    except:
        schools_frame = schools_frame.append(['NaN'], ignore_index=True)

schools_frame.drop(0, axis=1, inplace = True)

CPU times: user 30min 12s, sys: 1min 37s, total: 31min 50s
Wall time: 32min 16s


In [75]:
col_names=[]
for element in list(schools_frame):
    name = 'schools/0/'+element;
    col_names.append(name)
dic = dict(zip(list(schools_frame), col_names))

schools_frame.rename(columns=dic, inplace=True)
schools_frame.shape

(100000, 11)

In [128]:
result = result.join(schools_frame)
result.shape

(100000, 124)

#### Итерация 7

In [77]:
%%time
personal_frame = pd.DataFrame()
for personal in profiles['personal']:
    try:
        personal_frame = personal_frame.append(json_normalize(personal), ignore_index=True)
    except:
        personal_frame = personal_frame.append(['NaN'], ignore_index=True)

personal_frame.drop(0, axis=1, inplace = True)

CPU times: user 20min 5s, sys: 49.5 s, total: 20min 55s
Wall time: 21min 10s


In [78]:
col_names=[]
for element in list(personal_frame):
    name = 'personal/'+element;
    col_names.append(name)
dic = dict(zip(list(personal_frame), col_names))

personal_frame.rename(columns=dic, inplace=True)
personal_frame.shape

(100000, 8)

In [127]:
result = result.join(personal_frame)
result['personal/langs']=result['personal/langs'].str.join(sep=',')
result.shape

(100000, 113)

#### Итерация 8

In [83]:
%%time
relatives_frame = pd.DataFrame()
for relatives in profiles['relatives']:
    try:
        relatives_frame = relatives_frame.append(json_normalize(relatives[0]), ignore_index=True)
    except:
        relatives_frame = relatives_frame.append(['NaN'], ignore_index=True)

relatives_frame.drop(0, axis=1, inplace = True)

CPU times: user 12min 7s, sys: 13.6 s, total: 12min 21s
Wall time: 12min 22s


In [84]:
col_names=[]
for element in list(relatives_frame):
    name = 'relatives/'+element;
    col_names.append(name)
dic = dict(zip(list(relatives_frame), col_names))

relatives_frame.rename(columns=dic, inplace=True)
relatives_frame.shape

(100000, 3)

In [126]:
result = result.join(relatives_frame)
result.shape

(100000, 105)

#### Итерация 9

In [86]:
%%time
universities_frame = pd.DataFrame()
for universities in profiles['universities']:
    try:
        universities_frame = universities_frame.append(json_normalize(universities[0]), ignore_index=True)
    except:
        universities_frame = universities_frame.append(['NaN'], ignore_index=True)

universities_frame.drop(0, axis=1, inplace = True)

CPU times: user 34min 16s, sys: 1min 7s, total: 35min 24s
Wall time: 35min 26s


In [120]:
col_names=[]
for element in list(universities_frame):
    name = 'universities/0/'+element;
    col_names.append(name)
dic = dict(zip(list(universities_frame), col_names))

universities_frame.rename(columns=dic, inplace=True)
universities_frame.shape

(100000, 11)

In [141]:
result = result.join(universities_frame)
result.shape

(100000, 127)

### Результат нормализации
В результате нормализации получили переменнюу @result, из которой удаляем не нужные нам фичи

In [143]:
drop_cols = ['blacklisted_by_me', 'can_post', 'can_see_all_posts', 'can_see_audio',
'can_send_friend_request', 'can_write_private_message', 'first_name',
'friend_status', 'home_phone', 'is_favorite', 'is_friend',
'is_hidden_from_feed', 'last_name', 'maiden_name', 'photo_100', 'photo_200',
'photo_200_orig', 'photo_400_orig', 'photo_50', 'photo_id', 'photo_max',
'photo_max_orig', 'screen_name','city', 'blacklisted', 'country', 'deactivated',
'domain', 'last_seen', 'occupation', 'personal', 'relatives', 'schools', 'career', 'universities', 'crop_photo']

In [147]:
result.drop(drop_cols, axis=1, inplace = True)

In [148]:
result.head()

,about,activities,bdate,books,common_count,education_form,education_status,facebook,facebook_name,faculty,faculty_name,followers_count,games,graduation,has_mobile,has_photo,home_town,id,instagram,interests,livejournal,mobile_phone,movies,music,nickname,online,online_app,online_mobile,quotes,relation,relation_partner,sex,site,skype,status,status_audio,tv,twitter,university,university_name,verified,wall_comments,city/id,city/title,universities/0/chair,universities/0/chair_name,universities/0/city,universities/0/country,universities/0/education_form,universities/0/education_status,universities/0/faculty,universities/0/faculty_name,universities/0/graduation,universities/0/id,universities/0/name,relatives/id,relatives/type,relatives/name,personal/alcohol,personal/langs,personal/life_main,personal/people_main,personal/religion,personal/smoking,personal/inspired_by,personal/political,schools/0/city,schools/0/class,schools/0/country,schools/0/id,schools/0/name,schools/0/speciality,schools/0/year_from,schools/0/year_graduated,schools/0/year_to,schools/0/type,schools/0/type_str,occupation/id,occupation/name,occupation/type,carreer/0/city_id,carreer/0/company,carreer/0/country_id,carreer/0/from,carreer/0/position,carreer/0/until,carreer/0/group_id,last_seen/platform,last_seen/time,country/id,country/title
0,NaN,NaN,18.11,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,32047.0,NaN,NaN,1.0,1,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,,0,NaN,NaN,NaN,NaN,NaN,2,http://twitter.com/#!/perekopsky,NaN,,NaN,NaN,perekopsky,NaN,NaN,0.0,1.0,1.0,Москва,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"ООО ""В Контакте""",work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.526767e+09,1.0,Россия
1,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,55695.0,NaN,NaN,1.0,1,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,,0,NaN,NaN,NaN,NaN,NaN,2,https://telegram.org,NaN,Some people choose to see the ugliness in this...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,СПбГУ,university,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.524989e+09,1.0,Россия
2,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,10882.0,NaN,NaN,1.0,1,NaN,10,alexkuz,NaN,NaN,NaN,NaN,NaN,,0,NaN,NaN,NaN,NaN,NaN,2,,NaN,Ratio Putiniana - victoria Rutenica.,NaN,NaN,itsmekuzya,NaN,NaN,0.0,1.0,2.0,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Replika,work,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,1.526771e+09,1.0,Россия
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,21.10.1982,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,4615.0,NaN,NaN,1.0,1,NaN,17,NaN,NaN,NaN,,NaN,NaN,Викторович,0,NaN,NaN,NaN,NaN,NaN,2,http://vk.com/id17,NaN,But if anything should happen we'll be sure to...,NaN,NaN,NaN,NaN,NaN,0.0,1.0,2.0,Санкт-Петербург,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68.0,СПбГЭТУ (ЛЭТИ),university,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.526775e+09,1.0,Россия


In [151]:
result['bdate/day'] = result['bdate'].str.split('.').str.get(0)
result['bdate/month'] = result['bdate'].str.split('.').str.get(1)
result['bdate/year'] = result['bdate'].str.split('.').str.get(2)
result.drop('bdate', axis=1, inplace = True)

In [160]:
result.shape

(100000, 93)

### Записываем полученную таблицу в чистовой CSV-файл для удобство дальнейшей работы

In [154]:
#result.to_csv('normalized_input_data.csv')

In [156]:
#!ls -l